In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
import ipywidgets as widgets
from matplotlib.patches import Rectangle

In [2]:
# binning raw data
def spikes_bin_data(spike_times_raw_data, spike_clusters_data):
    spike_times_raw_data = np.squeeze(spike_times_raw_data)
    spike_clusters_data = np.squeeze(spike_clusters_data)
    spike_times_sec = spike_times_raw_data / 3e4 # convert from 30khz samples to seconds
    # set up bin edges - 20 ms here
    bins_seconds = np.arange(np.min(spike_times_sec), np.max(spike_times_sec), 0.02)
    # make list of lists for spike times specific to each cluster
    spikes = [spike_times_sec[spike_clusters_data == cluster] for cluster in np.unique(spike_clusters_data)]
    # bin
    binned_spikes = []
    for cluster in spikes:
        counts, _ = np.histogram(cluster, bins_seconds)  
        binned_spikes.append(counts)
    binned_spikes = np.array(binned_spikes) # should be [#neurons, #bins]
    return binned_spikes

In [3]:
# if prev and post aren't exact intervals, flash warning and find nearest bin
def spikes_binned_event_average(binned_spikes, event_start, event_ids, bin_size_sec=0.02, window_start_sec = 0.1, window_end_sec = 0.5):
    bintime_prev = int(window_start_sec * 50)
    bintime_post = int(window_end_sec * 50 + 1)
    windowsize = bintime_prev + bintime_post
    bin_size = bin_size_sec * 1000

    # To bin: divide by 20, floor
    stim_binned = np.floor(event_start * 1000 / bin_size).astype(int)
    stim_binned = np.transpose(stim_binned)


    u_stim_ids = np.unique(event_ids)

    # Initialize final_avg matrix
    final_avg = np.empty((binned_spikes.shape[0], len(u_stim_ids), windowsize))

    for neuron_id in range(binned_spikes.shape[0]):

        for stim_id in u_stim_ids:
            stim_indices = np.where(event_ids[0] == stim_id)[0]

            neuron_stim_data = np.empty((len(stim_indices), windowsize))
            
            for i, stim_idx in enumerate(stim_indices):
                bin_id = int(stim_binned[0][stim_idx])
                selected_columns = binned_spikes[neuron_id, bin_id - bintime_prev: bin_id + bintime_post]
                neuron_stim_data[i,:] = selected_columns

            bin_average = np.mean(neuron_stim_data, axis=0)/bin_size_sec
            final_avg[neuron_id, int(stim_id) - 1, :] = bin_average
    return final_avg


In [4]:
from pathlib import Path
dpath = Path(r'../../data/')
st_samp = np.load(dpath / 'spike_times.npy')
sc = np.load(dpath / 'spike_clusters.npy')
binned_spikes = spikes_bin_data(st_samp,sc)
event_start = np.load("../../data/natImsOnTimes.npy")
event_ids = np.load("../../data/natImsIDs.npy") 
test = spikes_binned_event_average(binned_spikes,event_start,event_ids)

In [ ]:
# Static graph below for all stimuli per neuron
# for i in range(0,10):
#     y = test[36][i]
#     x = np.arange(-100, 520, step=20)
#     plt.plot(x,y)



# plt.xlabel('Time from stimulus onset (20 ms bins)')

# plt.ylabel('Number of Spikes Per Second')
# plt.title('Neuron [36] Spiking Activity with Respect to Each Stimulus')

# # Add the gray rectangle
# plt.axvspan(0, 300, color='gray', alpha=0.3)

# plt.show()


In [ ]:
# Static graph for all neruons per stimuli
# for i in range(0,753):
#     y = test[i][9]
#     x = np.arange(-100, 520, step=20)
#     plt.plot(x,y)
   


# plt.xlabel('Time from Stimulus ID [0] display (20 ms bins)')

# plt.ylabel('Number of Spikes Per Second')
# plt.title('Neuron Spiking Activity with Respect to Stimulus ID [0]')

# # Add the gray rectangle
# plt.axvspan(0, 300, color='gray', alpha=0.3)

# plt.show()

In [ ]:
def plot_event_average_interaction(): #wrapper function
    
#graph: slider for time, dropdown for which id to show

In [8]:
def slope_viz_stimuli_per_neuron(t=-100, neuron_id = 355):
    
    # Plotting data:
    for i in range(0,test.shape[1]):
        y = test[neuron_id][i]
        x = np.arange(-100, 520, step=20)
        plt.plot(x,y)

    # Labels:
    plt.xlabel('Time from stimulus onset')
    plt.ylabel('Number of Spikes Per Second')
    plt.title(f'Neuron {neuron_id} Spiking Activity with Respect to Each Stimulus')

    #Accessories:
    plt.axvspan(0, 300, color='gray', alpha=0.3)
    plt.axvline(t, color='red', linestyle='--',)
    # Set y-axis limits
     # Calculate y-axis limits
    max_y = max([max(test[neuron_id][i]) for i in range(10)])  # Maximum y-value across all lines
    if max_y < 10:
        max_y = 10  # Set ymax to 10 if the default max is lower than 10
    plt.ylim(0, max_y)
   
    # plt.legend()
    plt.show()

In [5]:
def slope_viz_neurons_per_stimuli(t = -100, stim_id = 0):
    # Plotting data:
    for i in range(0,test.shape[0]):
        y = test[i][stim_id]
        x = np.arange(-100, 520, step=20)
        plt.plot(x,y)
    
    # Labels:
    plt.xlabel(f'Time from Stimulus {stim_id} display (20 ms bins)')
    plt.ylabel('Number of Spikes Per Second')
    plt.title(f'Neuron Spiking Activity with Respect to Stimulus ID {stim_id}')

    # Accessories:
    plt.axvspan(0, 300, color='gray', alpha=0.3)
    plt.axvline(t, color='red', linestyle='--',)

    plt.show()

In [6]:
time_slider = widgets.IntSlider(value=-100, min=-100, max=500, step=5, description='Time')
time_slider.layout.width = '6.55in'
time_slider.layout.margin = '0 -15px'

neuron_dropdown = widgets.Dropdown(
    options= range(0,test.shape[0]),
    value=355,
    description='Neuron ID:',
)
neuron_dropdown.layout.margin = "20px 20px"

stimuli_dropdown = widgets.Dropdown(
    options= range(0,test.shape[1]),
    value=0,
    description='Stimulus ID:',
)
stimuli_dropdown.layout.margin = "20px 20px"

# select_view_button = widgets.RadioButtons(
#     options=[("Stimulus View", "stim"), ("Neuron View", 'neuron')],
#     value = 'stim',
#     description='Select View:',
# )

In [9]:
# Link the function with the interact function
output = widgets.interactive_output(slope_viz_stimuli_per_neuron, {'t': time_slider, 'neuron_id': neuron_dropdown})

# Display the widgets and the output
display(widgets.VBox([neuron_dropdown,time_slider]))
display(output)

Output()

In [7]:
output = widgets.interactive_output(slope_viz_neurons_per_stimuli, {'t': time_slider, 'stim_id': stimuli_dropdown})
# Display the widgets and the output
display(widgets.VBox([stimuli_dropdown,time_slider]))
display(output)

Output()

In [17]:
def plot_appropriate_graph(view="stim"):
    # plt.clf()  # Clear the current plot
    
    if view == "stim":
        output = widgets.interactive_output(slope_viz_neurons_per_stimuli, {'t': time_slider, 'stim_id': stimuli_dropdown})
        # Display the widgets and the output
        display(widgets.VBox([stimuli_dropdown,time_slider]))
        display(output)
    
    elif view == "neuron":
        # Link the function with the interact function
        output = widgets.interactive_output(slope_viz_stimuli_per_neuron, {'t': time_slider, 'neuron_id': neuron_dropdown})

        # Display the widgets and the output
        display(widgets.VBox([neuron_dropdown,time_slider]))
        display(output)

# Define the radio button widget
# select_view_button = widgets.RadioButtons(
#     options=[("Stimulus View", "stim"), ("Neuron View", "neuron")],
#     value='stim',
#     description='Select View:'
# )

# Observe changes in the radio button and trigger plot_appropriate_graph
# select_view_button.observe(lambda change: plot_appropriate_graph(change.new), names='value')

# # Display the radio button widget
# display(select_view_button)


In [19]:
plot_appropriate_graph("stim")

Output()